# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [ ]:
from crewai import Agent, Task, Crew
import os
from langchain_openai import ChatOpenAI

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [ ]:
import os
from ..utils import pretty_print_result

os.environ["OPENAI_API_KEY"] = 'dummy_key'
os.environ["SERPER_API_KEY"]  = '' # you can get free serper api keys from serper.dev. Just sign up and copy here.

llm = ChatOpenAI(
    model="ollama/llama3:latest",
    base_url="http://localhost:11434/v1"
)

## Creating Agents

In [30]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    llm=llm,
    allow_delegation=False,
    verbose=True
)

In [31]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    llm=llm,
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [32]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [33]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [34]:
from crewai.tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [35]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [36]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [37]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [38]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [39]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=True,
	memory=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [41]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 93673fdd-1617-42c7-bc03-d01492f28b8a                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

2025-07-06 11:57:48,495 - 132282793414784 - rag_storage.py-rag_storage:138 - ERROR: Error during short_term search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
2025-07-06 11:57:48,807 - 132282793414784 - rag_storage.py-rag_storage:138 - ERROR: Error during entities search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Sales Representative                                                                                    │
│                                                                                                                 │
│  Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that    │
│  recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile,   │
│  focusing on key decision-makers, recent business developments, and potential needs that align with our         │
│  offerings. This task is crucial for tailoring our engagement strategy effectively.                             │
│  Don't make assumptions and only use information you absolutely sure about.                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Sales Representative                                                                                    │
│                                                                                                                 │
│  Thought: Here's my thought process:                                                                            │
│  Thought: I need to start by gathering information about DeepLearningAI                                         │
│                                                                                                                 │
│  Using Tool: List files in directory                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{}"                                                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

/home/taher490/Desktop/crew_ai/myenv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/home/taher490/Desktop/crew_ai/myenv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

2025-07-06 11:58:12,385 - 132282793414784 - rag_storage.py-rag_storage:109 - ERROR: Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
2025-07-06 11:58:12,542 - 132282793414784 - rag_storage.py-rag_storage:109 - ERROR: Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'


╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: e7eb108a-ab09-4ec4-bd96-4bd1a577cad6                                                                     │
│  Agent: Lead Sales Representative                                                                               │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 93673fdd-1617-42c7-bc03-d01492f28b8a                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Subject: Enhance DeepLearningAI's Innovation with CrewAI Solutions                               │
│                                                                                                                 │
│  Dear Andrew,                                                                                                   │
│                                                                                                                 │
│  I hope this email finds you well. I wanted to reach out and congratulate you on the recent product launch at   │
│  DeepLearningAI. Your vision and leadership have been instrumental in shaping the company's growth, and it's    │
│  exciting to see your team's hard work come to fruition.                                                        │
│                                                                                                                 │
│  As a pioneer in AI education, I'm confident that our solutions can support your goals of driving innovation    │
│  and efficiency within your organization. Our expertise in integrating AI solutions with existing systems can   │
│  help streamline operations and drive innovation.                                                               │
│                                                                                                                 │
│  I'd like to schedule a meeting to discuss how CrewAI can support DeepLearningAI's recent product launch and    │
│  future initiatives. We can explore potential areas where our solutions can provide value, such as:             │
│                                                                                                                 │
│  * Strategic partnership: Tailored solutions that cater to your specific needs                                  │
│  * Integration: Seamless integration with existing systems to drive innovation                                  │
│  * Innovation: Complementary AI education and training solutions                                                │
│                                                                                                                 │
│  Please let me know if you're available for a meeting this week or next. I'm looking forward to exploring the   │
│  possibilities of collaboration between our organizations.                                                      │
│                                                                                                                 │
│  Best regards,                                                                                                  │
│                                                                                                                 │
│  [Your Name]                                                                                                    │
│                                                                                                                 │
│  Note: The sentiment analysis tool provided a positive sentiment, which is reflected in the tone of the email   │
│  draft.                                                                                                         │
│                                                       

- Display the final result as Markdown.

In [42]:
from IPython.display import Markdown
Markdown(result.raw)

Subject: Enhance DeepLearningAI's Innovation with CrewAI Solutions

Dear Andrew,

I hope this email finds you well. I wanted to reach out and congratulate you on the recent product launch at DeepLearningAI. Your vision and leadership have been instrumental in shaping the company's growth, and it's exciting to see your team's hard work come to fruition.

As a pioneer in AI education, I'm confident that our solutions can support your goals of driving innovation and efficiency within your organization. Our expertise in integrating AI solutions with existing systems can help streamline operations and drive innovation.

I'd like to schedule a meeting to discuss how CrewAI can support DeepLearningAI's recent product launch and future initiatives. We can explore potential areas where our solutions can provide value, such as:

* Strategic partnership: Tailored solutions that cater to your specific needs
* Integration: Seamless integration with existing systems to drive innovation
* Innovation: Complementary AI education and training solutions

Please let me know if you're available for a meeting this week or next. I'm looking forward to exploring the possibilities of collaboration between our organizations.

Best regards,

[Your Name]

Note: The sentiment analysis tool provided a positive sentiment, which is reflected in the tone of the email draft.